In [ ]:
!pip install transformers
!pip install datasets
# !pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import statistics
import re

import torch 
import torch.nn.functional as F
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

from tqdm.notebook import tqdm

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
df = pd.read_csv("/content/disagreement_single_BERT.csv", encoding= 'unicode_escape')
remove_list = ['\n>', '>']
remove_words = lambda x: ' '.join([word.lower().replace('\n','') for word in x.split() if word not in remove_list])

df['comment'] = df['comment'].apply(remove_words)
df

,comment,CODE
0,"i hear about how it sets you free, but jeeze, ...",0
1,"those are ""elected officials"", not public serv...",1
2,"the right of the people to keep and bear arms,...",1
3,.....so she had it coming? liberal hypocrite,0
4,'someone has to supply arms to terrorist creat...,1
...,...,...
1601,link to story (ny times) http://thelede.blogs....,0
1602,"first i've heard of approval voting, but i hav...",0
1603,"should give them to santorum, so he can get co...",0
1604,and why do you say that? curious because the r...,0


In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 256

EPOCHS = 3
LEARNING_RATE = 2e-5
DROP_OUT = 0.3

MODEL_NAME_OR_PATH = "bert-base-cased"

In [ ]:

train, test = train_test_split(df,test_size=0.1,random_state =2)
x_train, y_train = train['comment'].values.tolist(), train['CODE'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['CODE'].values.tolist()

print(train.shape)
print(test.shape)

(1445, 2)
(161, 2)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config =  BertConfig.from_pretrained(MODEL_NAME_OR_PATH)
config.num_labels = 1

model = BertForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH, config= config).to(device)
model.train()
model.dropout.p = DROP_OUT
train_encodings = tokenizer(x_train, truncation=True, padding=True,max_length =256,  return_tensors='pt')
test_encodings = tokenizer(x_test, truncation=True, padding=True,max_length =256, return_tensors='pt')
print(train_encodings['input_ids'].shape)
print(config.to_json_string())

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

torch.Size([1445, 256])
{
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [ ]:
class OLID_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):

    item = {key: torch.clone(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

def create_data_loader(x, y, batch_size):
    dataset = OLID_Dataset(x,y)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

train_dataset = OLID_Dataset(train_encodings,y_train)
test_dataset = OLID_Dataset(test_encodings,y_test)

In [ ]:
class Trainer():

  def __init__(
    self,
    model,
    train_dataset,
    eval_dataset,
    loss_function = None,
    loss_weights = None,
    batch_size = 16
  ):

    self.model = model
    self.tokenizer = tokenizer
    self.train_dataset= train_dataset
    self.eval_dataset = eval_dataset
    self.loss_function = loss_function 
    self.loss_weights = loss_weights
    self.metrics = {"Training": {}, "Eval": {}}
    self.batch_size = batch_size


  def compute_metrics(self, y_true, y_pred):
    precision, recall, fscore, support = score(y_true, y_pred)
    accuracy = accuracy_score(y_true,y_pred)
    return {
        "Accuracy" : accuracy,
        "Precision": precision,
        "Recall": recall,
        "Fscore": fscore,
        "Support": support,
        "Macro F1": f1_score(y_true, y_pred, average='macro')
    }

  def create_data_loader(self, dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

  def eval_op(self):
    y_true = []
    y_pred = []
    with torch.no_grad():
      for dl in self.test_dataloader:
        input_ids = dl['input_ids'].to(device)
        attention_mask = dl['attention_mask'].to(device)
        targets = dl['labels'].to(device).float()
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        y_true += targets.cpu().numpy().tolist()
        y_pred += nn.Sigmoid()(outputs.logits).round().squeeze().cpu().tolist()
      print('EVAL ACCURACY: {}'.format(accuracy_score(y_true,y_pred)),
            'EVAL F1: {}'.format(f1_score(y_true, y_pred, average='macro')))
    return (y_true, y_pred)

  def fit(self, epochs = 2, learning_rate = 2e-5, use_scheduler = True):

    self.train_dataloader = self.create_data_loader(self.train_dataset, self.batch_size)
    self.test_dataloader = self.create_data_loader(self.eval_dataset,self.batch_size)
    total_steps = len(self.train_dataloader)
    print('TOTAL STEPS:', total_steps)

    optimizer = AdamW(model.parameters(),  lr=learning_rate)
    if use_scheduler:
      scheduler = get_linear_schedule_with_warmup(optimizer, total_steps, (epochs-1) * total_steps)

    for epoch in range(epochs):
      step = 0 
      losses = []
      y_true = []
      y_pred = []
      print("EPOCH {}".format(epoch+1))
      for dl in tqdm(self.train_dataloader, total=len(self.train_dataloader), desc="Training... "):
        optimizer.zero_grad()
        step += 1
        input_ids = dl['input_ids'].to(device)
        attention_mask = dl['attention_mask'].to(device)
        targets = dl['labels'].to(device).float()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        y_true += targets.cpu().numpy().tolist()
        y_pred += nn.Sigmoid()(outputs.logits).round().squeeze().cpu().tolist()
        loss = self.loss_function(outputs.logits, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

        if use_scheduler:
          scheduler.step()
    
        losses.append(loss.detach().cpu().item())
        if step%10 == 0:
          print("TRAINING LOSS: {}".format(statistics.mean(losses)),
                "TRAIN ACCURACY: {}".format(accuracy_score(y_true,y_pred)))
          eval_y_true, eval_y_pred = self.eval_op()
          model.train()

      self.metrics["Training"]["Epoch {}".format(epoch)] = self.compute_metrics(y_true, y_pred)
      self.metrics["Eval"]["Epoch {}".format(epoch)] = self.compute_metrics(eval_y_true, eval_y_pred)



In [ ]:
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([11]).to(device))
t = Trainer(model, train_dataset, test_dataset, loss_fn, batch_size=30)

In [ ]:
t.fit(epochs=10)

TOTAL STEPS: 49
EPOCH 1


TRAINING LOSS: 2.5273289918899535 TRAIN ACCURACY: 0.79
EVAL ACCURACY: 0.7763975155279503 EVAL F1: 0.5071428571428571
TRAINING LOSS: 2.418670654296875 TRAIN ACCURACY: 0.6866666666666666
EVAL ACCURACY: 0.4161490683229814 EVAL F1: 0.4134108527131783
TRAINING LOSS: 2.188987123966217 TRAIN ACCURACY: 0.5677777777777778
EVAL ACCURACY: 0.2422360248447205 EVAL F1: 0.22644927536231885
TRAINING LOSS: 2.0626709312200546 TRAIN ACCURACY: 0.49666666666666665
EVAL ACCURACY: 0.2608695652173913 EVAL F1: 0.24928490262920733

EPOCH 2


TRAINING LOSS: 1.514467430114746 TRAIN ACCURACY: 0.31666666666666665
EVAL ACCURACY: 0.32919254658385094 EVAL F1: 0.32854494902687675
TRAINING LOSS: 1.496472990512848 TRAIN ACCURACY: 0.36833333333333335
EVAL ACCURACY: 0.40372670807453415 EVAL F1: 0.40315106580166815
TRAINING LOSS: 1.4963595271110535 TRAIN ACCURACY: 0.39666666666666667
EVAL ACCURACY: 0.4472049689440994 EVAL F1: 0.4441168483531831
TRAINING LOSS: 1.467835745215416 TRAIN ACCURACY: 0.4275
EVAL ACCURACY: 0.5341614906832298 EVAL F1: 0.5196324143692564

EPOCH 3


TRAINING LOSS: 1.3499040365219117 TRAIN ACCURACY: 0.6066666666666667
EVAL ACCURACY: 0.42857142857142855 EVAL F1: 0.42678018575851395
TRAINING LOSS: 1.2996824026107787 TRAIN ACCURACY: 0.6066666666666667
EVAL ACCURACY: 0.6086956521739131 EVAL F1: 0.5801217038539553
TRAINING LOSS: 1.218389634291331 TRAIN ACCURACY: 0.6311111111111111
EVAL ACCURACY: 0.6459627329192547 EVAL F1: 0.617385866166354
TRAINING LOSS: 1.1381068140268327 TRAIN ACCURACY: 0.6616666666666666
EVAL ACCURACY: 0.7701863354037267 EVAL F1: 0.6722781537107334

EPOCH 4


TRAINING LOSS: 0.6760794341564178 TRAIN ACCURACY: 0.8666666666666667
EVAL ACCURACY: 0.6770186335403726 EVAL F1: 0.6186224489795917
TRAINING LOSS: 0.6937304183840751 TRAIN ACCURACY: 0.8533333333333334
EVAL ACCURACY: 0.782608695652174 EVAL F1: 0.6899928481047477
TRAINING LOSS: 0.7041605174541473 TRAIN ACCURACY: 0.84
EVAL ACCURACY: 0.7142857142857143 EVAL F1: 0.6453065134099617
TRAINING LOSS: 0.6576740995049477 TRAIN ACCURACY: 0.8583333333333333
EVAL ACCURACY: 0.782608695652174 EVAL F1: 0.6899928481047477

EPOCH 5


...

TRAINING LOSS: 0.36157115250825883 TRAIN ACCURACY: 0.94
EVAL ACCURACY: 0.7763975155279503 EVAL F1: 0.7100840336134453
TRAINING LOSS: 0.38437815234065054 TRAIN ACCURACY: 0.9266666666666666
EVAL ACCURACY: 0.8198757763975155 EVAL F1: 0.7479622132253712
TRAINING LOSS: 0.4016120612621307 TRAIN ACCURACY: 0.9344444444444444
EVAL ACCURACY: 0.7515527950310559 EVAL F1: 0.6872571872571872
TRAINING LOSS: 0.4436142690479755 TRAIN ACCURACY: 0.9075
EVAL ACCURACY: 0.6956521739130435 EVAL F1: 0.6426597816732347

EPOCH 7


TRAINING LOSS: 0.43766937255859373 TRAIN ACCURACY: 0.94
EVAL ACCURACY: 0.7329192546583851 EVAL F1: 0.6661522881805468
TRAINING LOSS: 0.40145469084382057 TRAIN ACCURACY: 0.93
EVAL ACCURACY: 0.782608695652174 EVAL F1: 0.7159348691838483
TRAINING LOSS: 0.356853744884332 TRAIN ACCURACY: 0.9366666666666666
EVAL ACCURACY: 0.8385093167701864 EVAL F1: 0.7403225806451613
TRAINING LOSS: 0.34363498240709306 TRAIN ACCURACY: 0.9425
EVAL ACCURACY: 0.8074534161490683 EVAL F1: 0.7254222368927765

EPOCH 8


TRAINING LOSS: 0.21871487498283387 TRAIN ACCURACY: 0.97
EVAL ACCURACY: 0.8322981366459627 EVAL F1: 0.7455662862159789
TRAINING LOSS: 0.2690494991838932 TRAIN ACCURACY: 0.9633333333333334
EVAL ACCURACY: 0.7763975155279503 EVAL F1: 0.7054878048780489
TRAINING LOSS: 0.2798014665643374 TRAIN ACCURACY: 0.9544444444444444
EVAL ACCURACY: 0.7701863354037267 EVAL F1: 0.6948414527944264
TRAINING LOSS: 0.270303725451231 TRAIN ACCURACY: 0.9525
EVAL ACCURACY: 0.8136645962732919 EVAL F1: 0.7368134263295554

EPOCH 9


TRAINING LOSS: 0.18748470395803452 TRAIN ACCURACY: 0.9766666666666667
EVAL ACCURACY: 0.84472049689441 EVAL F1: 0.7406070761100727
TRAINING LOSS: 0.20334476381540298 TRAIN ACCURACY: 0.9783333333333334
EVAL ACCURACY: 0.8198757763975155 EVAL F1: 0.7205028434600419
TRAINING LOSS: 0.21155654564499854 TRAIN ACCURACY: 0.9744444444444444
EVAL ACCURACY: 0.8198757763975155 EVAL F1: 0.7205028434600419
TRAINING LOSS: 0.20298570822924375 TRAIN ACCURACY: 0.975
EVAL ACCURACY: 0.8260869565217391 EVAL F1: 0.738999536822603

EPOCH 10


TRAINING LOSS: 0.188149194419384 TRAIN ACCURACY: 0.97
EVAL ACCURACY: 0.8260869565217391 EVAL F1: 0.7269864341085271
TRAINING LOSS: 0.20413440577685832 TRAIN ACCURACY: 0.97
EVAL ACCURACY: 0.8198757763975155 EVAL F1: 0.7267193444541997
TRAINING LOSS: 0.21099737361073495 TRAIN ACCURACY: 0.9688888888888889
EVAL ACCURACY: 0.7888198757763976 EVAL F1: 0.6896825396825397
TRAINING LOSS: 0.20793483071029187 TRAIN ACCURACY: 0.9708333333333333
EVAL ACCURACY: 0.8509316770186336 EVAL F1: 0.7659883720930233



In [ ]:
t.metrics

{'Eval': {'Epoch 0': {'Accuracy': 0.2608695652173913,
   'Fscore': array([0.15602837, 0.34254144]),
   'Macro F1': 0.24928490262920733,
   'Precision': array([1.        , 0.20666667]),
   'Recall': array([0.08461538, 1.        ]),
   'Support': array([130,  31])},
  'Epoch 1': {'Accuracy': 0.5341614906832298,
   'Fscore': array([0.6031746 , 0.43609023]),
   'Macro F1': 0.5196324143692564,
   'Precision': array([0.96610169, 0.28431373]),
   'Recall': array([0.43846154, 0.93548387]),
   'Support': array([130,  31])},
  'Epoch 2': {'Accuracy': 0.7701863354037267,
   'Fscore': array([0.85140562, 0.49315068]),
   'Macro F1': 0.6722781537107334,
   'Precision': array([0.8907563 , 0.42857143]),
   'Recall': array([0.81538462, 0.58064516]),
   'Support': array([130,  31])},
  'Epoch 3': {'Accuracy': 0.782608695652174,
   'Fscore': array([0.85943775, 0.52054795]),
   'Macro F1': 0.6899928481047477,
   'Precision': array([0.89915966, 0.45238095]),
   'Recall': array([0.82307692, 0.61290323]),
  